In [1]:
import pandas as pd
import sqlite3
from sqlite3 import Error
pd.options.mode.chained_assignment = None # not to get SettingWithCopyWarning

## Set Variables - Ticker, CIK etc

In [2]:
cik = '0000051253'
ticker = 'IFF'

## Function to use

In [3]:
def write_new_data_into_sqltable(db_path=str, df=pd.DataFrame, statement=str, cik=str):
    # create connection
    connection = None
    try:
        connection = sqlite3.connect(db_path)
        print("Connection to SQLite DB successful!")
    except Error as e:
        print(f"The error '{e}' occurred.")
    
    # extend cik to 10 character
    df['cik'] = [str(cik).zfill(10) for cik in df['cik'].values]
    # keep only one CIK in DataFrame
    df = df.loc[df.cik == cik]
    if len(df) == 0:
        print('CIK filter removed every row.')
    # check CIK is in database
    if cik not in pd.read_sql_query("SELECT DISTINCT cik FROM {}".format(statement), connection).values:
        # get newest report date
        new_updated_date = pd.Timestamp(df['end'].values.max()).normalize() + pd.Timedelta(days=1)
        # create direct input tables with strings
        ingest_df = df[pd.read_sql_query("PRAGMA table_info({});".format(statement), connection).name.values].copy()
        for column in ingest_df.columns:
            ingest_df[column].apply(str)
        print('Creation of input database is successful!')
        # write dataframe into table
        ingest_df.to_sql(
            name=statement,
            con=connection,
            if_exists='append',
            index=False)
        print('Data is written into database!')
        # update main table's update column
        cursor = connection.cursor()
        try:
            cursor.execute("UPDATE main SET updated = '{}' WHERE cik = '{}'".format(new_updated_date, cik))
            connection.commit()
            cursor.close()
            print("Main table's updated column freshed successfully! Cursor closed.")
        except Error as e:
            print(f"The error '{e}' occurred")
    else:
        print('Writing into the database is unsuccessful due to existing CIK.')
    # close connection
    connection.close()
    print('Connection closed!')

In [4]:
def add_extra_data_to_sqltable(db_path=str, df=pd.DataFrame, statement=str, cik=str):
    # create connection
    connection = None
    try:
        connection = sqlite3.connect(db_path)
        print("Connection to SQLite DB successful!")
    except Error as e:
        print(f"The error '{e}' occurred.")
    
    # extend cik to 10 characters
    df['cik'] = [str(cik).zfill(10) for cik in df['cik'].values]
    # check CIK is in main datatable
    if cik in pd.read_sql_query("SELECT DISTINCT cik FROM main", connection)['cik'].values:    
        # get last update date relating to company CIK number
        last_update = pd.Timestamp(pd.read_sql_query("SELECT updated FROM main WHERE cik = '{}'".format(cik), connection)['updated'].values[0])
        # filter input df & keep only new data
        filtered_df = df.loc[df.end > last_update]
        filtered_df.reset_index(inplace=True, drop=True)
        if len(filtered_df) != 0 and str(last_update) not in pd.read_sql_query("SELECT end FROM {} WHERE cik = '{}'".format(statement, cik), connection)['end'].values:
            # create direct input tables with strings
            ingest_df = filtered_df[pd.read_sql_query("PRAGMA table_info({});".format(statement), connection).name.values].copy()
            for column in ingest_df.columns:
                ingest_df[column].apply(str)
            print('Creation of input database is successful!')
            # write dataframe into table
            ingest_df.to_sql(
                name=statement,
                con=connection,
                if_exists='append',
                index=False)
            print('Data is written into database!')
            # update main table's update column
            cursor = connection.cursor()
            try:
                new_updated_date = str(pd.to_datetime('today').normalize())
                cursor.execute("UPDATE main SET updated = '{}' WHERE cik = '{}'".format(new_updated_date, cik))
                connection.commit()
                cursor.close()
                print("Main table's updated column freshed successfully! Cursor closed.")
            except Error as e:
                print(f"The error '{e}' occurred")
        else:
            print('There is NO new data!')
    else:
        print('CIK is not in the main data table! Use another function!')
    # close connection
    connection.close()
    print('Connection closed!')

## 0. Create company row in main table & Load X-CHECKED data

In [5]:
data = pd.read_csv('../sec_report_csv/{}_{}_sec_reports.csv'.format(cik, ticker), parse_dates=['end'])
data.head()

,end,form,cik,rev,cost_sale,sale_gen_adm,inpairm,op_income,int_income,int_exp,...,other_ass,tot_ass,short_debt,acc_pay,other_curr_liab,tot_curr_liab,tot_long_debt,other_liab,tot_equity,ticker
0,2018-06-30,10-Q,51253,920016000,521299000,157407000,0.0,154509000,20655000,53246000,...,157017000,4673442000,6500000,315656000,263448000,699370000,1717189000,238635000,1749855000,IFF
1,2018-09-30,10-Q,51253,907548000,506882000,157796000,0.0,159268000,4158000,23914000,...,167978000,9615542000,45985000,312236000,257364000,758456000,4331242000,235332000,4031814000,IFF
2,2018-12-31,10-Q,51253,907548000,506882000,157796000,0.0,159268000,4158000,23914000,...,288673000,12889395000,48642000,471382000,530508000,1128311000,4504417000,248436000,6032951000,IFF
3,2018-12-31,10-K,51253,3977539000,2294832000,707461000,0.0,583882000,35243000,132558000,...,288673000,12889395000,48642000,471382000,530508000,1128311000,4504417000,248436000,6032951000,IFF
4,2019-03-31,10-Q,51253,1297402000,766143000,213182000,0.0,163870000,7278000,36572000,...,583389000,13211420000,84003000,476413000,414626000,1144134000,4421430000,492029000,6143211000,IFF


## 1. Write new data into SQL table

In [10]:
for statement in ['income_statement', 'balance_sheet']:
    print('Strrting to fill {} table in SQL database!'.format(statement))
    write_new_data_into_sqltable(
        db_path='../database/statements.db',
        df=data,
        statement=statement,
        cik=cik
    )

Strrting to fill income_statement table in SQL database!
Connection to SQLite DB successful!
Calc date
Creation of input database is successful!
Data is written into database!
Main table's updated column freshed successfully! Cursor closed.
Connection closed!
Strrting to fill balance_sheet table in SQL database!
Connection to SQLite DB successful!
Writing into the database is unsuccessful due to existing CIK.
Connection closed!


## 2. Update Existing company with newer data

In [11]:
for statement in ['income_statement', 'balance_sheet']:
    print('Strrting to update {} table in SQL database!'.format(statement))
    add_extra_data_to_sqltable(
        db_path='../database/statements.db',
        df=data,
        statement=statement,
        cik=cik
    )

Strrting to update income_statement table in SQL database!
Connection to SQLite DB successful!
There is NO new data!
Connection closed!
Strrting to update balance_sheet table in SQL database!
Connection to SQLite DB successful!
There is NO new data!
Connection closed!
